# <font color='red'>Problem Definition & Formulation</font>


The problem formulation phase of the ML Pipeline is critical, and it’s where everything begins.It starts by seeing a problem and thinking “what question, if I could answer it, would provide the most value to my business?” 

Part of the problem formulation phase includes seeing where there are opportunities to use machine learning and consider the following questions:
1.	Is machine learning appropriate for this problem, and why or why not?
2.	What is the ML problem if there is one, and what would a success metric look like?
3.	What kind of ML problem is this?
4.	Is the data appropriate?


# Answers for the four mentioned questions!
To answer those question, I must understand the problem well and then explore the data to build and inution about it, then come up with rigoures arguments which lead to a conclusion and prespictives. 

1)	ML is appropriate because of the scale and Variety of the Data . There are potentially High diemensional Features and about of half million of the Training examples which  makes the problem to be very difficult for Human to be solved without the use of ML. In addition, ML solution will offer a scalable and reusable solution for the problem.

2)	The problem is :How well can we predict the future sale price of a bulldozer, given its characteristics and previous examples of how much similar bulldozers have been sold for?
i.	Success would be the minimum root mean squared log error (RMSLE) because it is a Kaggle competition and  Kaggle has set this evaluation metric to being used.

3)	This is a supervised Regression ML problem because we have a labeled data point and the output is a Numerical value.

4)	This data is appropriate because it has variety of historical data of similar products and The characteristics of the Bulldozer,  and there are a lot of examples with  labeled target to train, tune and test the model .


# <font color='blue'>Importing Libraries and Modules </font>

In [ ]:
# Import data analysis tools 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

%matplotlib inline


# import model modules
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRFRegressor
# import model evaluation modules
from sklearn.metrics import mean_squared_log_error #because kaggle want this metric



# <font color='blue'>Loading Data </font>

There are 3 datasets:

* Train.csv - Historical bulldozer sales examples up to 2011 (close to 400,000 examples with 50+ different attributes, including SalePrice which is the target variable).
* Valid.csv - Historical bulldozer sales examples from January 1 2012 to April 30 2012 (close to 12,000 examples with the same attributes as Train.csv).
* Test.csv - Historical bulldozer sales examples from May 1 2012 to November 2012 (close to 12,000 examples but missing the SalePrice attribute, as this is what we'll be trying to predict).

## Features


For this dataset, Kaggle provide a data dictionary which contains information about what each attribute of the dataset means. You can download this file directly from the Kaggle competition page  [here](https://www.kaggle.com/c/bluebook-for-bulldozers/download/Bnl6RAHA0enbg0UfAvGA%2Fversions%2FwBG4f35Q8mAbfkzwCeZn%2Ffiles%2FData%20Dictionary.xlsx) (account required) or view it on Google Sheets.

With all of this being known, let's get started!



In [ ]:
# Import the training and validation set
df = pd.read_csv("../input/bluebook-for-bulldozers/TrainAndValid.csv")

# <font color='red'>  EDA & Data Preparation </font>

# EDA is crucial step that help us explore and understand our data to build an intution  about it, and outline the required preprocessing steps before the modeling stage. 

## * In this new version, I will use Pandas profiling as an automated EDA technique.

In [ ]:
# Automated_EDA_Report = ProfileReport (df)


In [ ]:
# Automated_EDA_Report.to_notebook_iframe()

In [ ]:
# Lets start with an overview of our data
# df.head(10)

In [ ]:
#Lets identify our features(Columns) names, counts and datatypes
#df.info()

### It is clear that we have combination of numerical and categorical data, and we have missed data because the count of features is not the same for each features.

In [ ]:
#Lets check the missing values  
#df.isna().sum()

### There are some features without missing data, some with accepted small amount of missing data that can be imputed, and others with very large missing data that we can not deal with and we will drop them later.

In [ ]:
#Check the distribution of the SalePrice using histogram
#df.SalePrice.plot.hist()

In [ ]:
# Check some statistics about SalePrice
#df["SalePrice"].describe()

## Parsing dates
When working with time series data, it's a good idea to make sure any date data is the format of a [datetime object](https://docs.python.org/3/library/datetime.html) (a Python data type which encodes specific information about dates).


In [ ]:
df = pd.read_csv("../input/bluebook-for-bulldozers/TrainAndValid.csv",
                 low_memory=False,
                 parse_dates=["saledate"])

In [ ]:
# check the datatype of saledate after parsing
#df.info()

### As saledate is datetime object type, We can make some feature engineering on it to extract the different attributes of It.

In [ ]:
# lets sort the date first
df.sort_values(by=['saledate'],
               inplace=True,
               ascending=True)

In [ ]:
df["saleYear"] = df.saledate.dt.year
df["saleMonth"] = df.saledate.dt.month
df["saleDay"] = df.saledate.dt.day
df["saleDayofweek"] = df.saledate.dt.dayofweek
df["saleDayofyear"] = df.saledate.dt.dayofyear

# Drop original saledate as we do not need it
df.drop("saledate", axis=1, inplace=True)

### Handling missing data
we will start by recheck the miising data to drop columns with significant amount of missied data

## After some experimentation, I found that imputing the missing values with the median has good impact in the performance of model, so I will not drop them in this version.

In [ ]:
#df.isna().sum()

In [ ]:
# lets drop columns with more than 50 % missed data because this columns(Features) do not add any value
# as well they may mislead the algorithm and to save the memory and processing time 
#limitPer = len(df) * .50
#df = df.dropna(thresh=limitPer,axis=1)

In [ ]:
#df.info()

# Filling the missed value by Median for numerical features and for Categorical non numerical Features after converting them into numerical types

In [ ]:
# Fill numeric rows with the median
for label, content in df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # Fill missing numeric values with median since it's more robust than the mean
            df[label] = content.fillna(content.median())

In [ ]:
# Turn categorical variables into numbers
for label, content in df.items():
    # Check columns which *aren't* numeric
    if not pd.api.types.is_numeric_dtype(content):
        # We add the +1 because pandas encodes missing categories as -1
        df[label] = pd.Categorical(content).codes+1        

In [ ]:
# Check for Missing Values
#df.isna().sum()

### It turns out that There is no missing values now, all data is numerical form,  So lets Go to Modeling stage to build up a predictive model for that regression problem!

# <font color='red'>  Modeling </font>

In [ ]:
# Splitting data ito x and y and into train and valid datasets
df_val = df[df.saleYear == 2012]
df_train = df[df.saleYear != 2012]
X_train, y_train = df_train.drop("SalePrice", axis=1), df_train.SalePrice
X_valid, y_valid = df_val.drop("SalePrice", axis=1), df_val.SalePrice

np.random.seed(42) # random seed for reproduciblity of the result.

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape


### According to the Kaggle data page, the validation set and test set are split according to dates.
* Training = all samples up until 2011
* Valid = all samples form January 1, 2012 - April 30, 2012
* Test = all samples from May 1, 2012 - November 2012

In [ ]:
#After experimentation, I will use the RandomForesst Regressor which gives me a high performance in this dataset
RF = RandomForestRegressor(n_estimators=50,  n_jobs=-1, random_state=42,)
XGB= XGBRFRegressor(n_estimators=50, n_jobs=-1, random_state=42)

In [ ]:
 RF.fit(X_train, y_train)
 XGB.fit(X_train, y_train)   

## Model Evaluation 

In [ ]:
#lets score on the validation set to see our performance regarding to the leaderbooard on kaggle
RF.score(X_valid, y_valid)
XGB.score(X_valid, y_valid)


### Score by default use the R2 metric, but the competition choosed RMSLE
* Root Mean Square Log Error

In [ ]:
# there is no direct method to get RMSLE, so we need to find MSLE first
y_preds = RF.predict(X_valid)
y_preds2 = XGB.predict(X_valid)

RMSLE = np.sqrt(mean_squared_log_error(y_valid, y_preds))
print (f"RMSLE of RandomeForest on Valid Dataset is:{RMSLE}")
RMSLE2 = np.sqrt(mean_squared_log_error(y_valid, y_preds2))
print (f"RMSLE of XGB Regressor on Valid Dataset is:{RMSLE2}")

## Now, lets score on the test set to see our final performance
* we will use the same preprocessing steps that used on the training and validation sets, otherwise, we can not used our trained model!!

In [ ]:
X_test = pd.read_csv("../input/bluebook-for-bulldozers/Test.csv",
                 low_memory=False,
                 parse_dates=["saledate"])


In [ ]:
#sort by saledate
x_test.sort_values(by=['saledate'],
               inplace=True,
               ascending=True)

#feature engineering on Date
X_test["saleYear"] = X_test.saledate.dt.year
X_test["saleMonth"] = X_test.saledate.dt.month
X_test["saleDay"] = X_test.saledate.dt.day
X_test["saleDayofweek"] = X_test.saledate.dt.dayofweek
X_test["saleDayofyear"] = X_test.saledate.dt.dayofyear

# Drop original saledate as we do not need it
X_test.drop("saledate", axis=1, inplace=True)

In [ ]:
# Fill numeric rows with the median
for label, content in X_test.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # Fill missing numeric values with median since it's more robust than the mean
            X_test[label] = content.fillna(content.median())

In [ ]:
# Turn categorical variables into numbers
for label, content in X_test.items():
    # Check columns which *aren't* numeric
    if not pd.api.types.is_numeric_dtype(content):
        # We add the +1 because pandas encodes missing categories as -1
        X_test[label] = pd.Categorical(content).codes+1        

In [ ]:
#predicting the SalePrice
y_preds = RF.predict(X_test)


### Since we do not have the True Price and we can not submit  our solution on Kaggle, we can not calculate the final performance of the model (RMSLE )

In [ ]:
 #rmsle = np.sqrt(mean_squared_log_error(y_test, y_preds))
 #rmsle